# Angry tweets
## Dominik Tomkiewicz 140793
## Wojciech Rzeczycki 140770

W ramach drugiego projektu z Zaawansowanej eksploracji danych przygotowano model predykcji klasyfikacji dla sentymentu tweetów.
Analizie poddany został zbiór danych kilku tysięcy tweetów.

# 1. Wykorzystane biblioteki

In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
import csv
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# 2. Pobranie danych

Dane składają się z wpisów o tweetach: sentymentu użytkownika (pozytywna/negatywna/neutralna) oraz jego wpisu. Wszystkie słowa potratkowano jako stringi, dodatkowo ze zbioru danych testowych usunięto wartości puste występujące na końcu pliku.

Stworzono algorytm, który klasyfikuje tweety do odpowiedniego sentymentu.

In [2]:
classes = ['negative', 'positive', 'neutral']
stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 
              'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 
              'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 
              'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 
              'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 
              'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 
              'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 
              'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 
              'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 
              'doing', 'it', 'how', 'further', 'was', 'here', 'than']

tweets = pd.read_csv("train.csv", header=0, sep=',', dtype = str)

temp = np.random.rand(len(tweets)) < 0.80
tweetsTrain = tweets[temp]
tweetsTest = tweets[~temp]

tweetsTrainData = []
tweetsTrainLabel = []

tweetsTestData = []
tweetsTestLabel = []

In [3]:
tweetsTrain.head(6)

,Id,Category,Tweet
0,635769805279248384,negative,Not Available
1,635930169241374720,neutral,IOS 9 App Transport Security. Mm need to check...
2,635950258682523648,neutral,"Mar if you have an iOS device, you should down..."
3,636030803433009153,negative,@jimmie_vanagon my phone does not run on lates...
4,636100906224848896,positive,Not sure how to start your publication on iOS?...
5,636176272947744772,neutral,"Two Dollar Tuesday is here with Forklift 2, Qu..."


In [4]:
tweetsTest.head(6)

,Id,Category,Tweet
6,636276311560859648,neutral,If you're not already signed up to test my iOS...
8,636356154151575552,neutral,YouTube Gaming Launches Tomorrow with iOS and ...
9,636360240921972736,neutral,@astrill Yashan from BBC @bbcchinese the VPN a...
13,636724154260176896,positive,"Met with iOS Developer today. We may have a ""g..."
19,637085524683976704,positive,Siri knows all about #Apple's iOS event on the...
20,637103428494188548,positive,"Who's ready for 0.12.1 Build 12 Tomorrow, Migh..."


# 3. Przetwarzanie tweetów

Dla każdego wpisu w listach treningowej i testowej dokonujemy transformacji danych:

- usuwamy słowa zaczynające się od symbolu @ (służące do oznaczenia użytkownika, co nie dostarcza informacji na temat sentymentu),

- usuwamy słowa znajdujące się w zmiennej stop_words (słowa łączące wypowiedzi, nie dostarczające przydatnych informacji),

- po oczyszczeniu danych dodajemy do listy sentyment i zawartość tweeta (co posłuży jego klasyfikacji).

In [5]:
for index, tweet in tweetsTrain.iterrows():
    try:
        tweet[2] = re.sub(r'@\w+', '', tweet[2])
        querywords = tweet[2].split()
        resultwords  = [word for word in querywords if word.lower() not in stop_words]
        tweet[2] = ' '.join(resultwords)
        tweetsTrainData.append(tweet[2])
        tweetsTrainLabel.append(tweet[1])
    except:
        print(type(tweet))
            
for index, tweet in tweetsTest.iterrows():
    try:
        tweet[2] = re.sub(r'@\w+', '', tweet[2])
        querywords = tweet[2].split()
        resultwords  = [word for word in querywords if word.lower() not in stop_words]
        tweet[2] = ' '.join(resultwords)
        tweetsTestData.append(tweet[2])
        tweetsTestLabel.append(tweet[1])
    except:
            print(type(tweet))

# 4. Przypisanie wag słowom

Dla tweetów cechy, które pomogą w klasyfikacji to słowa, które w nich występują, a ich wagi to częstotliwość ich występowania w tweetach.

Za pomocą metody TfidfVectorizer transformujemy dane wejściowe w wektory dla klasyfikatora ustawiając odpowiednie parametry:

- min_df - odrzucane są słowa występujące w mniej niż określonej liczbie wpisów,

- max_df - odrzuca słowa pojawiające się w określonym ułamku dokumentów,

- sublinear_tf i use_idf - używanie sublinear weighting i wagowania IDF.

Za pomocą funkcji fit_transform zostanie stworzony słownik słowo-waga. Funkcja transform transformuje dane do wersji słownikowej.

In [6]:
vectorizer = TfidfVectorizer(min_df = 7,
                             max_df = 0.70,
                             sublinear_tf=True,
                             use_idf=True)

train_vectors = vectorizer.fit_transform(tweetsTrainData)
test_vectors = vectorizer.transform(tweetsTestData)

# 5. Trening

Po analizie możliwości klasyfikatorów dostarczonych przez Scikit-learn zdecydowano się na wykorzystanie algorytmu Naiwnego Bayesa. Poniżej kod trenujący klasyfikator.

In [7]:
classifierNB = MultinomialNB()
classifierNB.fit(train_vectors, tweetsTrainLabel)
predictionNB=classifierNB.predict(test_vectors)

In [8]:
print("Wyniki dla naiwnego Bayesa")
print(classification_report(tweetsTestLabel, predictionNB))

Wyniki dla naiwnego Bayesa
              precision    recall  f1-score   support

    negative       0.61      0.11      0.19       194
     neutral       0.49      0.37      0.42       416
    positive       0.59      0.86      0.70       556

    accuracy                           0.56      1166
   macro avg       0.56      0.45      0.44      1166
weighted avg       0.56      0.56      0.51      1166



# 6. Testowanie

Poniżej kod testujący działanie klasyfikatora:

In [9]:
tweetsTrain = pd.read_csv("train.csv", header = 0, sep=',', dtype = str)
tweetsTest = pd.read_csv("test.csv", header = 0, sep=',', dtype = str).dropna()

tweetsTrainData = []
tweetsTrainLabel = []

tweetsTestId = []
tweetsTestData = []
tweetsTestLabel = []

for index, tweet in tweetsTrain.iterrows():
    tweet[2] = re.sub(r'@\w+', '', tweet[2])
    querywords = tweet[2].split()
    resultwords  = [word for word in querywords if word.lower() not in stop_words]
    tweet[2] = ' '.join(resultwords)
    tweetsTrainData.append(tweet[2])
    tweetsTrainLabel.append(tweet[1])
    
for index, tweet in tweetsTest.iterrows():
    tweet[1] = re.sub(r'@\w+', '', tweet[1])
    querywords = tweet[1].split()
    resultwords  = [word for word in querywords if word.lower() not in stop_words]
    tweet[1] = ' '.join(resultwords)
    tweetsTestData.append(tweet[1])
    tweetsTestId.append(tweet[0])
    
vectorizer = TfidfVectorizer(min_df = 7,
                             max_df = 0.70,
                             sublinear_tf=True,
                             use_idf=True)

train_vectors = vectorizer.fit_transform(tweetsTrainData)
test_vectors = vectorizer.transform(tweetsTestData)

classifierNB = MultinomialNB()
classifierNB.fit(train_vectors, tweetsTrainLabel)
predictionNB=classifierNB.predict(test_vectors)

header = {'Id': tweetsTestId, 'Category': predictionNB}
result = pd.DataFrame(data = header)
result.to_csv('result.csv', sep = ',', columns=['Id', 'Category'], index=False)